<a href="https://colab.research.google.com/github/blindauth/abstention_experiments/blob/master/hema/AnalyzeHemeResults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -r abstention
!pip uninstall abstention
!git clone https://github.com/blindauth/abstention.git
!pip install abstention/
![[ -e abstention_experiments ]] || git clone https://github.com/blindauth/abstention_experiments.git

rm: cannot remove 'abstention': No such file or directory
Cloning into 'abstention'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 51 (delta 18), reused 42 (delta 9), pack-reused 0
Unpacking objects: 100% (51/51), done.
Processing ./abstention
  Stored in directory: /tmp/pip-ephem-wheel-cache-bduzkq6s/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
Cloning into 'abstention_experiments'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 1894 (delta 4), reused 5 (delta 0), pack-reused 1873
Receiving objects: 100% (1894/1894), 173.29 MiB | 33.97 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [3]:
!ls abstention_experiments/hema

heme_task0_abstention_results.json.gz  record_30_model_vo6bz_preds
heme_task1_abstention_results.json.gz  record_3_model_e0JNc_preds
record_12_model_bo6Z3_preds	       record_6_model_T8zn7_preds
record_15_model_Bscof_preds	       record_9_model_8pkw7_preds
record_18_model_xZ1BL_preds	       Run_Heme_Experiments.ipynb
record_21_model_ovIXJ_preds	       test_labels.txt.gz
record_24_model_7n9bs_preds	       valid_labels.txt.gz
record_27_model_3QpKH_preds


In [0]:
import gzip
import json

task0_abstention_results =\
  json.loads(gzip.open(
    "abstention_experiments/hema/heme_task0_abstention_results.json.gz").read())
task1_abstention_results =\
  json.loads(gzip.open(
    "abstention_experiments/hema/heme_task1_abstention_results.json.gz").read())

In [7]:
task0_abstention_results.keys()

dict_keys(['metricname_to_abstfrac_to_abstncalib_to_perf', 'metricname_to_baselineperfs'])

In [10]:
task0_abstention_results['metricname_to_abstfrac_to_abstncalib_to_perf']['auroc']['0.15'].keys()

dict_keys(['distfrom0.5:none', 'distfrom0.5:platt', 'estwindowdelta_auroc_15pc:none', 'estwindowdelta_auroc_15pc:platt', 'estwindowdelta_auroc_30pc:none', 'estwindowdelta_auroc_30pc:platt', 'fumera_auroc_15pc:none', 'fumera_auroc_15pc:platt', 'fumera_auroc_30pc:none', 'fumera_auroc_30pc:platt', 'fumera_tpratfpr_15pc:none', 'fumera_tpratfpr_15pc:platt', 'fumera_tpratfpr_30pc:none', 'fumera_tpratfpr_30pc:platt', 'js:none', 'js:platt', 'mcwindowdelta_tpratfpr_15pc:none', 'mcwindowdelta_tpratfpr_15pc:platt', 'mcwindowdelta_tpratfpr_30pc:none', 'mcwindowdelta_tpratfpr_30pc:platt', 'uncertainty:none', 'uncertainty:platt'])

In [16]:
from abstention.figure_making_utils import (
    wilcox_srs, get_ustats_mat,
    get_tied_top_and_worst_methods)
import numpy as np

metricnames = ['auroc', 'tpratfpr_0.2']
abstfracs = ['0.15', '0.3']

abstfrac_to_metricname_to_methodnames = {
    '0.15': {
        'auroc': ['distfrom0.5:none',
                  'distfrom0.5:platt',
                  #'js:none',
                  'js:platt',
                  #'estwindowdelta_auroc_15pc:none',
                  'estwindowdelta_auroc_15pc:platt',
                  'fumera_auroc_15pc:none',
                  'fumera_auroc_15pc:platt'],
        'tpratfpr_0.2': ['distfrom0.5:none',
                         'distfrom0.5:platt',
                         #'js:none',
                         'js:platt',
                         #'mcwindowdelta_tpratfpr_15pc:none',
                         'mcwindowdelta_tpratfpr_15pc:platt',
                         'fumera_tpratfpr_15pc:none',
                         'fumera_tpratfpr_15pc:platt']
    },
    '0.3': {
        'auroc': ['distfrom0.5:none',
                  'distfrom0.5:platt',
                  #'js:none',
                  'js:platt',
                  #'estwindowdelta_auroc_30pc:none',
                  'estwindowdelta_auroc_30pc:platt',
                  'fumera_auroc_30pc:none',
                  'fumera_auroc_30pc:platt'],
        'tpratfpr_0.2': ['distfrom0.5:none',
                         'distfrom0.5:platt',
                         #'js:none',
                         'js:platt',
                         #'mcwindowdelta_tpratfpr_30pc:none',
                         'mcwindowdelta_tpratfpr_30pc:platt',
                         'fumera_tpratfpr_30pc:none',
                         'fumera_tpratfpr_30pc:platt']
    }   
}

for abstention_results in [task0_abstention_results,
                           task1_abstention_results]:
  for metricname in metricnames:
    for abstfrac in abstfracs:
      methodnames = abstfrac_to_metricname_to_methodnames[abstfrac][metricname]
      nsamples = len(list(
        abstention_results['metricname_to_abstfrac_to_abstncalib_to_perf']
                          [metricname][abstfrac].values())[0])
      print(metricname, abstfrac)
      ustats_mat = get_ustats_mat(
          method_to_perfs=
            abstention_results['metricname_to_abstfrac_to_abstncalib_to_perf']
                [metricname][abstfrac],
          method_names=methodnames,
          max_ustat=nsamples*(nsamples+1)/2 )
      tied_top_methods, tied_worst_methods =(
          get_tied_top_and_worst_methods(
              ustats_mat,
              methodnames,
              #0.05 threshold for one-sided test from normal approx when N=30,
              #from https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
              threshold=151
          ))
      tied_top_methods = [methodnames[x] for x in tied_top_methods]
      print(tied_top_methods)

auroc 0.15
['estwindowdelta_auroc_15pc:platt']
auroc 0.3
['estwindowdelta_auroc_30pc:platt']
tpratfpr_0.2 0.15
['mcwindowdelta_tpratfpr_15pc:platt']
tpratfpr_0.2 0.3
['mcwindowdelta_tpratfpr_30pc:platt']
auroc 0.15
['estwindowdelta_auroc_15pc:platt']
auroc 0.3
['estwindowdelta_auroc_30pc:platt']
tpratfpr_0.2 0.15
['mcwindowdelta_tpratfpr_15pc:platt']
tpratfpr_0.2 0.3
['mcwindowdelta_tpratfpr_30pc:platt']
